<a href="https://colab.research.google.com/github/BSpekter/Special-Public/blob/master/Fashion_MNIST_Bayburin_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torchvision import datasets,transforms
from torch import nn, optim
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt

from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

Как говорил Лектор, наверное стоит внести ссылки на источники, ибо так не совсем научно. Постараемся так и сделать.



Большая благодарность чувствуется вот тут:

https://towardsdatascience.com/building-neural-network-using-pytorch-84f6e75f9a

https://towardsdatascience.com/training-a-neural-network-using-pytorch-72ab708da210

Часть кода реализовано от этого источника 
https: //www.arunprakash.org/2018/12/cnn-fashion-mnist-dataset-pytorch.html

Вдохновение пару раз посещало и на этом сайте:
https://www.kaggle.com/swapnil2195/part-4-fashion-mnist-solution/data

Красивый код временами созерцался тут: https://necromuralist.github.io/In-Too-Deep/posts/nano/pytorch/part-4-fashion/

Вот это точно можно потом посмотреть: https://github.com/mayurbhangale/fashion-mnist-pytorch/blob/master/CNN_Fashion_MNIST.ipynb

CNN ресурс тут: https://github.com/kishankg/2-Layer-CNN-with-Pytorch-Fashion-MNIST-/blob/master/2%20Layer%20CNN%20with%20Pytorch%20(Fashion-MNIST).ipynb

In [0]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])

In [0]:
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = True, transform=transform, download=True)

In [0]:
trainloader = torch.utils.data.DataLoader(trainset, shuffle=True, batch_size=128)

In [0]:
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', train = False, transform=transform, download = True)

In [0]:
testloader = torch.utils.data.DataLoader(testset, shuffle=False, batch_size=128)

Попробовал поиграть с обычной полносвязной нейросетью. Конечно не 92% Accuracy, но тоже ничего. И конечно стоит сказать что Dropout помогает.

Да и попробовал применить Adam and SGD.

In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 256)
        self.l2 = nn.Linear(256, 128)
        self.l3 = nn.Linear(128,64)
        self.l4 = nn.Linear(64,10)
    def forward(self,x):
        #So here we are trying to flatten images along vector
        x = x.view(x.shape[0],-1)

        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.log_softmax(self.l4(x), dim=1)
        return x

In [0]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model=Model().to(device)
criterion = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [0]:
epoch = 30
train_losses, test_losses = [],[]
for e in range(epoch):
    train_loss = 0
    test_loss = 0
    accuracy = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                images, labels = images.to(device), labels.to(device)

                log_ps = model(images)
                prob = torch.exp(log_ps)
                top_probs, top_classes = prob.topk(1, dim=1)
                equals = labels == top_classes.view(labels.shape)
                accuracy += equals.type(torch.FloatTensor).mean()
                test_loss += criterion(log_ps, labels)
        model.train()
    print("Epoch: {}/{}.. ".format(e+1, epoch),
              "Training Loss: {:.3f}.. ".format(train_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    train_losses.append(train_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))

In [0]:
plt.plot(train_losses,label = "Train losses")
plt.plot(test_losses, label = "Test losses")
plt.title("Here we use SGD algorithm")
plt.legend()

In [0]:
optimizer = optim.Adam(model.parameters(), lr=0.003)

In [0]:
epoch = 30
train_losses, test_losses = [],[]
for e in range(epoch):
    train_loss = 0
    test_loss = 0
    accuracy = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                images, labels = images.to(device), labels.to(device)

                log_ps = model(images)
                prob = torch.exp(log_ps)
                top_probs, top_classes = prob.topk(1, dim=1)
                equals = labels == top_classes.view(labels.shape)
                accuracy += equals.type(torch.FloatTensor).mean()
                test_loss += criterion(log_ps, labels)
        model.train()
    print("Epoch: {}/{}.. ".format(e+1, epoch),
              "Training Loss: {:.3f}.. ".format(train_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    train_losses.append(train_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))

In [0]:
plt.plot(train_losses,label = "Train losses")
plt.plot(test_losses, label = "Test losses")
plt.title("Here we use Adam algorithm")
plt.legend()

**Model with dropout**

In [0]:
class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(784, 256)
        self.l2 = nn.Linear(256, 128)
        self.l3 = nn.Linear(128,64)
        self.l4 = nn.Linear(64,10)
        
        self.dropout = nn.Dropout(p=0.2)
    def forward(self,x):
        x = x.view(x.shape[0],-1)
        
        x = self.dropout(F.relu(self.l1(x)))
        x = self.dropout(F.relu(self.l2(x)))
        x = self.dropout(F.relu(self.l3(x)))
        x = F.log_softmax(self.l4(x), dim=1)
        return x

In [0]:
model = Model().to(device)
optimizer = optim.Adam(model.parameters(),lr=0.003)

In [0]:
epoch = 30
train_losses, test_losses = [],[]
for e in range(epoch):
    train_loss = 0
    test_loss = 0
    accuracy = 0
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        train_loss += loss.item()

        loss.backward()
        optimizer.step()
    else:
        with torch.no_grad():
            model.eval()
            for images,labels in testloader:
                images, labels = images.to(device), labels.to(device)

                log_ps = model(images)
                prob = torch.exp(log_ps)
                top_probs, top_classes = prob.topk(1, dim=1)
                equals = labels == top_classes.view(labels.shape)
                accuracy += equals.type(torch.FloatTensor).mean()
                test_loss += criterion(log_ps, labels)
        model.train()
    print("Epoch: {}/{}.. ".format(e+1, epoch),
              "Training Loss: {:.3f}.. ".format(train_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))
    train_losses.append(train_loss/len(trainloader))
    test_losses.append(test_loss/len(testloader))

In [0]:
plt.plot(train_losses,label = "Train losses")
plt.plot(test_losses, label = "Test losses")
plt.title("Adam model with Dropout")
plt.legend()

Как видно, модель с Dropout ведёт себя лучше, пусть и не значительно.

Если я правильно понимаю, то дальше мы докажем что полносвязные сети проигрывают CNN в точности работы в задаче "классификации картинок".

**Попытаемся построить CNN**

In [0]:
num_epochs = 30
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [0]:
class CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [0]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
def test_and_plot(model, num_epochs, criterion, optimizer):
    train_loss, test_loss = [], []
    for e in range(num_epochs):
        start_time = time.time()

        trloss = 0
        teloss = 0

        model.train(True)
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)
            #Run the model
            output = model(images)
            loss = criterion(output, labels)
            trloss += loss.item()

            #Backprop and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.train(False)
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            output = model(images)
            loss = criterion(output, labels)
            teloss += loss.item()

        train_loss.append(trloss/len(trainloader))
        test_loss.append(teloss/len(testloader))

        #Printing results for the epoch
        print("Epoch {} of {} took {:.3f}s".format(e+1, num_epochs,
                                                  time.time()-start_time))
        print("Training loss in this epoch: \t{:.6f}".format(train_loss[-1]))

    plt.plot(train_loss, label="Train loss")
    plt.plot(test_loss, label="Test loss")
    plt.legend()

In [0]:
def test_accuracy(model):
    model.eval()  # eval mode (batchnorm uses moving mean/variance instead of mini-batch mean/variance)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in testloader:
            images, labels = images.to(device), labels.to(device)
            images = Variable(images.float())
            labels = Variable(labels)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print('Test Accuracy of the model: {} %'.format(100 * correct / total))

In [0]:
 test_and_plot(model, 30, criterion, optimizer)

In [0]:
test_accuracy(model)

Думаю тут можно сказать что lr для этой модели слишком высок.

Теперь попробуем модифицировать модель добавив Dropout и Batch normalization.

In [0]:
class CNN(nn.Module):
    def __init__(self,num_classes=10):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc = nn.Linear(7*7*32, 10)
        self.Dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.Dropout(F.relu(self.fc(out)))
        return out
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
test_and_plot(model, 30, criterion, optimizer)

In [0]:
test_accuracy(model)

Решили проблему - потеряли Accuracy. Видимо в дальнейшем нужно просто модифицировать параметры.

В общем случае поняли что добавленные нами методы работают. Сейчас придумаем новую модель для достижения Accuracy > 92%.

Буду надеятся что ключ к упеху - Дропауты, Батчнорм и больше слоёв.

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            )
        self.layer2 = nn.Sequential(
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.layer3 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.fc1 = nn.Linear(128*49, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

        self.LogSoftmax = nn.LogSoftmax( dim=1)

        self.Dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.Dropout(self.layer2(out))
        out = self.layer3(out)

        out = out.reshape(out.size(0), -1)

        out = F.relu(self.fc1(out))
        out = self.Dropout(self.fc2(out))
        out = self.LogSoftmax(self.fc3(out))
        return out
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
test_and_plot(model, 30, criterion, optimizer)

In [0]:
test_accuracy(model)

Цели по Accuracy достигли, попробуем сделать лучше.

In [0]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Sequential(
            nn.BatchNorm2d(1),
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer2 = nn.Sequential(
            nn.BatchNorm2d(16),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer3 = nn.Sequential(
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU()
            )
        self.layer4 = nn.Sequential(
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
            )
        self.fc1 = nn.Linear(25088, 256)
        self.fc2 = nn.Linear(256, 10)

        self.LogSoftmax = nn.LogSoftmax(dim=1)
        self.Dropout = nn.Dropout(p=0.3)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.Dropout(self.layer2(out))
        out = self.Dropout(self.layer3(out))
        out = self.Dropout(self.layer4(out))

        out = out.reshape(out.size(0), -1)

        out = F.relu(self.fc1(out))
        out = self.LogSoftmax(self.fc2(out))
        return out
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [0]:
test_and_plot(model, 30, criterion, optimizer)

In [0]:
test_accuracy(model)

Получилось не сильно лучше.
Думаю в целом задание выполнено.